### 0. Installing necessary packages

In [ ]:
!pip install networkx
!pip install geopandas
!pip install osmnx
!pip install pulp

> After the code block above finished running, you will need to re-start the run time (Runtime -> Restart runtime). After that, you can proceed with the next code blocks.

In [ ]:
# import packages
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
import numpy as np
from pulp import *

### 1. Open Street Maps
In this use case, we make use of Open Street Maps (OSM) to create and analyze real-world street networks. Since we are on a truck, we will be downloading a network with drivable routes in the heart of Amsterdam.
https://www.openstreetmap.org/

In [ ]:
### create graph with a location name
place = "Amsterdam"
# Transportation mode
mode = "drive"
# Create network graph
G = ox.graph_from_address(place, dist=3000, simplify=True, network_type=mode)    # 3000 meters
# Plot the network graph
fig, ax = ox.plot_graph(G);
plt.tight_layout()
plt.show()

In [ ]:
### create graph with latitude & longitude
coordinates = (52.3372232, 4.8729058)   # Accenture ITO Office
# Transportation mode
mode = "drive"
# create network graph
G_from_coordinates = ox.graph_from_point(coordinates, dist=3000, simplify=True, network_type=mode)
# Plot the network graph
fig, ax = ox.plot_graph(G_from_coordinates);
plt.tight_layout()
plt.show()

#### Network Properties

In [ ]:
# number of nodes and edges
print('Number of nodes:', len(G.nodes))
print('Number of edges:', len(G.edges))

In [ ]:
# View all nodes
G.nodes()

In [ ]:
# View all edges (for undirected graphs)
G.edges()

In [ ]:
# View both in- and out-edges (for directed graphs)
node = '***'
print('in edges:', G.in_edges(node))
print('out edges:', G.out_edges(node))

In [ ]:
# get attributes related to two nodes
G.get_edge_data('***', '***')

#### Try it yourself

In [ ]:
# 1. Find the coordinates of the node ID 46484771

### PUT YOUR CODE HERE ###

In [ ]:
# 2. How many streets connected to node 46294809 is one-way?

### PUT YOUR CODE HERE ###

In [ ]:
# 3. What is the maximum speed and street name(s) between nodes 46308817 and 46310809?

### PUT YOUR CODE HERE ###

### 2. Data Munipulation
Since the open street map only provides the distance between two nodes, the main goal of this section is to get the **traveling time** using columns **length** (in meters) and **maxspeed** (in km/h).


Handling the **maxspeed** column:
*   It is possible that, for some edges, the maximum speed is a list containing multiple values. In this case, **we will take the minimum speed**.
*   If the maximum speed is a null value, we will need to perform unit imputation. For simplicity, we assume the value to be **30 (km/h)**.


Also, bear in mind - although there is a maximum speed column in graph object, in real world situations traffic can never reach the maximum speed. Therefore, We will need to create a column as an approximated average speed so we could calculate the traveling time between two nodes.

<div>
<img src="https://www.researchgate.net/publication/332660949/figure/fig5/AS:751781516677120@1556250089722/Map-of-average-traffic-speeds-in-central-Amsterdam-Source-Spotzi.png" width="600">
</div>


*Source: researchgate.net/figure/Map-of-average-traffic-speeds-in-central-Amsterdam-Source-Spotzi_fig5_332660949*

In [ ]:
# get geoDF of nodes and edges
nodes, edges = ox.graph_to_gdfs(G)

In [ ]:
# print the first 5 rows of the nodes df
nodes.head()

In [ ]:
# print the first 5 rows of the edges df
edges.head()

In [ ]:
# for some rows, a list (e.g. [30, 50]) is stored in maxspeed column. choose the min in the list
edges['maxspeed'] = [min(np.array(i).astype(int)) if type(i)==list else i for i in edges['maxspeed']]

# assume max speed is 30 km/h
edges['maxspeed'] = [30 if pd.isnull(i) else int(i) for i in edges['maxspeed']]

#  calculate the average speed given the maximum
edges['avg_speed'] = [15 if pd.isnull(i) else \
                      15 if int(i) == 30 else \
                      30 if int(i) == 50 else \
                      int(i) for i in edges['maxspeed']
                      ]
 
# calculate the no. of minutes required to pass through the path (length devided by speed)
edges['time_minutes'] = (edges['length'] / 1000) / edges['avg_speed'] * 60

In [ ]:
# inspect the edges df and new columns
edges[['length', 'maxspeed', 'avg_speed', 'time_minutes']]

In [ ]:
# create network graphs from updated nodes and edges
G = ox.graph_from_gdfs(nodes, edges)

### 3. Linear Programming and Travel Salesman Problem (TSP)

#### **Linear Programming**
Linear programming is an optimization technique given a system of linear constraints and a linear objective. It is the most common type of mathematical programming model, and is widely used in industries such as transportation, telecommunications, energy, and manufacturing for planning, scheduling, routing and other resource-dependent use cases.

![picture](https://drive.google.com/uc?id=1Wa38aybLG5vFZjUPEY2WJyrIh8qjii-E)

When you construct a linear programming problem, there are three things you need to define:
* **Decision Variables**. The values to be computed by the solvers.
* **Objective function**. An objective that you want to optimize (i.e. maximize or minimize).
* **Constraints**. The restrictions of the model / scarcity of resources.


> From a practical point of view, what are the objective and constraints?



#### **Travel Salesman Problem**
Travel Salesman Problem, or TSP, is a special type of linear programming problems that are considered computationally expensive (see table) and requires [heuristics](https://en.wikipedia.org/wiki/Heuristic). It is an important topic in the field of operational research and route optimization, and is generalized to other optimization formulations such as the vehicle routing problem.


Here you can see the number of possible combinations given *n* locations (in an *undirected graph*).

| # locations | # Possible combinations |
| --- | --- |
| 3 | 2! = 2 |
| 4 | 3! = 6 |
| 5 | 24 |
| 6 | 120 |
| 7 | 720 |
| 8 | 5040 |
| 9  | 40320 |
| 10  |362880 |
| 11  |3628800 |
| 12 |39916800 |
| 13 |479001600 | 
| 14 |6227020800 |
| ... | |

Another important thing to consider in the problem formulation is the **sub-route elimination**. The main goal of the truck is to go through an complete route - from origin to destination - in one big loop, without an sub-tours in the solution (our truck cannot teleport!). 

Something we want to avoid:
<div>
   <img src="https://how-to.aimms.com/_images/Subtour.png" width="700">
</div>

*Source: https://how-to.aimms.com/Articles/332/332-Miller-Tucker-Zemlin-formulation.html*

In our use case, we used the Miller Tucker Zemlin formulation to ensure one complete route is returned, rather than an union of several sub routes.





#### Delivery Information

In [ ]:
# delivery locations - RAMEN!
addresses = [
             (52.3638643, 4.9344159),   # Betsubara Japanese Kitchen - Oost
             (52.3575133, 4.8995665),   # Fou Fow Ramen van Woustraat
             (52.3478993, 4.8915300),   # Takumi Ramen & Yakisoba
             (52.3871539, 4.9254829),   # Little Saigon Noord
             (52.3788579, 4.8850906),   # HINATA AMSTERDAM
             (52.3661336, 4.8983335),   # wagamama Rembrandtplein
             (52.3766845, 4.9015363),   # Ramen-Kingdom
             (52.3720887, 4.8957213),   # Ramen-ya
             (52.3676046, 4.8730433)    # OTEMBA Ramen
             ]

# specify start and end points
breakdown_coordinates = (52.36355, 4.87031)
service_point_coordinates = (52.36636, 4.92165)

# for simplicity, get the closest node in the map as the drop off point
all_locations_coordinates = [breakdown_coordinates] + addresses + [service_point_coordinates]
all_locations = [ox.nearest_nodes(G, coord[1], coord[0]) for coord in all_locations_coordinates]
breakdown = all_locations[0]
service_point = all_locations[-1]

# create a colour map
colour_map = ['blue' if i in all_locations[1:-1] else \
              'red' if i == breakdown else \
              'yellow' if i == service_point else \
              'grey' for i in list(G.nodes())
              ]

# node size
node_size = [200 if i in all_locations else \
             10 for i in list(G.nodes())
             ]

# plot coloured network
fig, ax = ox.plot_graph(
    G, node_color = colour_map, node_size = node_size
)

In [ ]:
# create travel time matrix (in minutes)
time_matrix = []

for i, origin_node in enumerate(all_locations):
  matrix_row = []
  for j, dest_node in enumerate(all_locations):
    # calculate the shortest path (Dijkstra's algorithm)
    time = nx.shortest_path_length(G, source=all_locations[i], target=all_locations[j], weight='time_minutes')
    matrix_row.append(time)
  time_matrix.append(matrix_row)

time_matrix = np.array(time_matrix)
print(np.round(time_matrix,1))

#### Cost Matrix Adjustment 
**Cost Matrix Adjustment - Closed loop TSP**

In a classical TSP, the 'salesman' travels to all locations once without revisiting and eventually return to the original starting point. However, that is not applicable in our case, since we started in the coordinates where the cooler broke down and ended in the warehouse. Therefore, we need to have some adjustments to the cost matrix.


![picture](https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/GLPK_solution_of_a_travelling_salesman_problem.svg/330px-GLPK_solution_of_a_travelling_salesman_problem.svg.png)

We reduced the 11x11 matrix to 10x10 by:


1.   Removing 'Destination' in the row
2.   Removing 'Origin' in the column
3.   Moving the 'Destination' to the first column.

By doing so, the origin and destination both have an index 0; when TSP solves the problem, index 0 will serve as a proxy and therefore a close loop can be reached.
![picture](https://drive.google.com/uc?id=1GeoWDiZ4BL3VL9gEJwjkQs_gIomk80Z8)

**Cost Matrix Adjustment - Drop off time**

Also, we need to take into account the time we need to drop off goods at every location. Some locations might be centrally location, but it takes a significant amount of time to complete delivery. In that case, it might be ideal to deliver to other locations given the time constraint. Therefore, we add the drop off time into the cost matrix.

#### Main TSP model
With the time matrix, we can finally create the TSP model and attempt to find the optimal path.

In [ ]:
def solve_tsp (time_matrix, time_limit, drop_off_time_vector = None):
  """ Given a time cost matrix, the remaining time and optionally a drop off time vector, the function returns a PuLP model and the decision variables 
  """
  # adjusting cost matrix - We will use this in the algorithm (see explanation below)
  cost_matrix = np.array(time_matrix)
  cost_matrix = np.delete(cost_matrix, -1, 0)      # remove last row (from destination)
  cost_matrix = np.delete(cost_matrix, 0, 1)       # remove first column (to origin)
  cost_matrix = np.roll(cost_matrix, 1, axis=1)    # move last column to the first row

  # get number of locations
  location_count = len(cost_matrix)

  if drop_off_time_vector is not None:
    # shape the vector into the same shape as the cost matrix
      drop_off_time_matrix = [[0] + drop_off_time_vector] * location_count
      cost_matrix = cost_matrix + np.array(drop_off_time_matrix)
  
  # define model
  model = LpProblem('tsp', LpMaximize)

  # define variables
  x = LpVariable.dicts("x",((i,j) for i in range(location_count) for j in range(location_count)), cat='Binary')

  # add objective function (-1 to exclude destination)
  model += (
      lpSum(x[i,j]  for i in range(location_count) for j in range(location_count)) - 1,
      "Sum of locations",
  )

  # constraints - time limit (in minutes) - with drop-off
  model += (
      lpSum([cost_matrix[i,j] * x[i,j] for i in range(location_count) for j in range(location_count)]) <= time_limit,
      "Time limit"
  )

  # constraint - origin
  model += (
      lpSum([x[0,j] for j in range(location_count)]) == 1,
      "Constraint for origin - 1 path leaving 0"
  )

  # constraint - destination
  model += (
      lpSum([x[i,0] for i in range(location_count)]) == 1,
      "Constraint for destination"
  )

  # constraint - each location should only be visited max. once
  for j in range(1, location_count):
    model += lpSum(x[i,j] for i in range(location_count)) <= 1

  for i in range(1,location_count):
    # constraints - cannot visit itself
    model += x[i,i] == 0    
    # constraints - entering and leaving a location
    model += lpSum(x[i,j] for j in range(location_count)) == lpSum(x[c,i] for c in range(location_count))

  # subroute elimination variable
  u = LpVariable.dicts("u", (i for i in range(location_count)), lowBound=1, upBound=location_count)

  # eliminate subtour
  for i in range(location_count):
      for j in range(location_count):
          if i!=j and (i!=0 and j!=0):
              model += u[j] >= u[i] + 1 - 100*(1-x[i,j])

  # solve model
  status = model.solve()
  print('Status:',LpStatus[status], '\nObjective:', value(model.objective))

  # print the total time needed
  total_time = sum([cost_matrix[i,j] * value(x[i,j]) \
                    for i in range(location_count) \
                    for j in range(location_count)]
                  )
  print('Total Time Required: {} minutes.'.format(total_time))

  return model, x

In [ ]:
# solve tsp and get the solution WITHOUT drop off time
model, variables = solve_tsp(time_matrix, time_limit = 70)

In [ ]:
# solve tsp and get the solution WITH drop off time
drop_off_time_vector = [5, 10, 15, 5, 10, 15, 5, 10, 15]
model, variables = solve_tsp(time_matrix, time_limit = 70, drop_off_time_vector = drop_off_time_vector)

#### Visualization

In [ ]:
def get_optimal_route(all_locations, variables):
  """ Given the decision variables, this function generates an ordered optimal route """

  # get location count
  location_count = len(all_locations)

  # create column label of adjusted cost matrix
  col_labels = ["destination"] + list(range(1, location_count-1)) 

  # get all the travelling pairs
  route = [(i, j) for i in range(location_count-1) for j in range(location_count-1) if value(variables[i,j])==1]

  # get the full list of routes
  dct = {x:y for x,y in route}
  ordered_route = ['origin']
  val = 0

  while True:
    val = dct[val]
    ordered_route.append(col_labels[val])
    if val == 0:
      break
  
  return(ordered_route)

In [ ]:
# use the function to find optimal route
optimal_route = get_optimal_route(all_locations, variables)
optimal_route

In [ ]:
def visualize_optimal_route(all_locations, variables):
  """ This function visualizes the optimal route """
  # get the corrsponding index in the original location matrix
  orig_matrix_indices = [0 if i == 'origin' else len(all_locations)-1 if i == 'destination' else i for i in optimal_route]
  optimal_node_paths = []

  for i in range(len(optimal_route)-1):
    # get the optimal path between two nodes in the ordered route list
    optimal_node_paths.append(
        nx.shortest_path(G, source=all_locations[orig_matrix_indices[i]], target=all_locations[orig_matrix_indices[i+1]], weight='time_minutes')
    )

  # plot map with optimal route
  fig, ax = ox.plot_graph_routes(
      G, 
      routes = optimal_node_paths,
      node_color = colour_map, 
      node_size = node_size, 
      route_colors = 'yellow'
      )

# plot
visualize_optimal_route(all_locations, variables)

#### Try it youself

In [ ]:
# 1. Assume the time limit is now 50 minutes without drop off time. How many locations can you deliver before going back to the warehouse?

### PUT YOUR CODE HERE ###

In [ ]:
# 2. Assume the time limit is now 50 minutes with the drop off time vector as [15, 6, 32, 15, 11, 4, 8, 25, 7]. 
# How many locations can you deliver before going back to the warehouse?

### PUT YOUR CODE HERE ###

In [ ]:
# 3. Same as Q2, but assume the time limit is now 120 minutes. How many locations can you now deliver?

### PUT YOUR CODE HERE ###

### Future Improvements
What do you think can be the next steps for this project?